# ML 11 - Object detection and localization with YOLO

**Objectives**: Learn about object detection using YOLO ( ("you only look once") deep Conv Neural Network (CNN).

**References**: The ideas presented in this lab  are described in two YOLO papers: 

- Joseph Redmon, Santosh Divvala, Ross Girshick, Ali Farhadi - [You Only Look Once: Unified, Real-Time Object Detection](https://arxiv.org/abs/1506.02640) (2015)
- Joseph Redmon, Ali Farhadi - [YOLO9000: Better, Faster, Stronger](https://arxiv.org/abs/1612.08242) (2016)

The implementation here use many components from Allan Zelener's github repository:
[YAD2K: Yet Another Darknet 2 Keras](https://github.com/allanzelener/YAD2K)

The pretrained weights used in this lab came from the official YOLO website: https://pjreddie.com/darknet/yolo/) 


- YOLO is a state-of-the-art object detection model suitable for on-line implementation.
- It runs an input image through a CNN which outputs a 19x19x5x85 dimensional volume. 
- The encoding can be seen as a grid where each of the 19x19 cells contains information about 5 boxes.
- Non-max suppression is used to filter through all the boxes. Specifically: 
    - Filter 1: Get rid of any box for which the class "score" is less than a chosen threshold.
    - Filter 2: Intersection over Union (IoU) thresholding to eliminate overlapping boxes. 
- Training a YOLO model from randomly initialized weights takes a very long time and requires a large dataset as well as lot of computation, in this lab we use previously trained model parameters. 
   

In [1]:
#Import relevant libraries
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)


import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body

%matplotlib inline

Using TensorFlow backend.


**Important Note**: Note that Keras's backend is imported as K. This means to call a Keras function, you should use `K.function(...)`.

## 1 - Problem Statement

You are working on a self-driving car. First, you'd like to build a car detection system. To collect data, you've mounted a camera to the front of the car, to collect pictures of the road every few seconds while you drive around. 

<center>
<video width="400" height="200" src="nb_images/road_video_compressed2.mp4" type="video/mp4" controls>
</video>
</center>

<caption><center> Pictures taken from a car-mounted camera while driving around Silicon Valley, provided by drive.ai (https://www.drive.ai/) 
</center></caption>

All images are stored into a folder and labelled by drawing bounding boxes around every car found. Here's an example of what means image labeling and how the bounding boxes look like.

<img src="nb_images/box_label.png" style="width:400px;height:250;">
<caption><center> <u> **Figure 1** </u>: **Image labelling**<br> </center></caption>

Overall, there are 80 classes that YOLO has to recognize, the class label $c$ is represented as an 80-dimensional vector one component of which is 1 and the others are 0. 
 

## 2 - YOLO

YOLO ("you only look once") is a popular algoritm because it achieves high accuracy while also being able to run in real-time. This algorithm "only looks once" at the image in the sense that it requires only one forward propagation pass through the network to make predictions. After non-max suppression, it outputs the recognized objects and the bounding boxes.

### 2.1 - Model details

- The YOLO **input** is a batch of m RGB images of shape (m, 608, 608, 3). 
- The YOLO **output** is a list of bounding boxes along with the recognized classes. Each bounding box is represented by 6 numbers $(p_c, b_x, b_y, b_h, b_w, c)$ as explained above. If you expand $c$ into an 80-dimensional vector, each bounding box is then represented by 85 numbers. 

$(p_c, b_x, b_y, b_h, b_w, c= [c_1, c_2....c_{end}])$ 

We will use 5 anchor boxes. So you can think of the YOLO architecture as the following: IMAGE (m, 608, 608, 3) -> DEEP CNN -> ENCODING (m, 19, 19, 5, 85). Lets see what this encoding represents. 

<img src="nb_images/architecture.png" style="width:600px;height:300;">
<caption><center> <u> **Figure 2** </u>: **Encoding architecture for YOLO**<br> </center></caption>

If the center/midpoint of an object falls into a grid cell, that grid cell is responsible for detecting that object.

Since we are using 5 anchor boxes, each of the 19 x19 cells encodes information about 5 boxes. Anchor boxes are defined by their width and height.

For simplicity, the last two dimensions of the shape (19, 19, 5, 85) encoding are flatten. So the output of the Deep CNN is (19, 19, 425).

<img src="nb_images/flatten.png" style="width:600px;height:300;">
<caption><center> <u> **Figure 3** </u>: **Flattening the last two dimensions**<br> </center></caption>

For each anchor box (of each cell) the following element-wise product is computed and extract a probability that the box contains a certain class.

<img src="nb_images/probability_extraction.png" style="width:600px;height:300;">
<caption><center> <u> **Figure 4** </u>: **Find the class detected by each box**<br> </center></caption>

One way to visualize YOLO's output is to plot the bounding boxes that it outputs as shown in Fig.6:  

<img src="nb_images/anchor_map.png" style="width:200px;height:200;">
<caption><center> <u> **Figure 6** </u>: Each grid cell gives 5 anchor boxes. In total, the model predicts: 19x19x5 = 1805 boxes just by looking once at the image (one forward pass through the network)! Different colors denote different classes. <br> </center></caption>

In Fig.6 are plotted only boxes that the model had assigned a high probability to, but this is still too many boxes. 

The next two functions (yolo_filter_boxes() and yolo_non_max_suppresion ()) provide 2 filters 
to down the YOLO output to a much smaller number of detected objects.

### 2.2 - Filter 1 - with a threshold on class scores

Get rid of any box for which the class "score" is less than a chosen threshold. Low score means, the box is not very confident about detecting a class. 

The model gives a total of 19x19x5x85 numbers, with each box described by 85 numbers. It is convenient to rearrange the (19,19,5,85) (or (19,19,425)) dimensional tensor into the following variables:  
- `box_confidence`: tensor of shape $(19 \times 19, 5, 1)$ containing $p_c$ (confidence probability that there's some object) for each of the 5 boxes predicted in each of the 19x19 cells.
- `boxes`: tensor of shape $(19 \times 19, 5, 4)$ containing $(b_x, b_y, b_h, b_w)$ for each of the 5 boxes per cell.
- `box_class_probs`: tensor of shape $(19 \times 19, 5, 80)$ containing the detection probabilities $(c_1, c_2, ... c_{80})$ for each of the 80 classes for each of the 5 boxes per cell.



In [2]:
#FILTER 1

def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = .6):
    """Filters YOLO boxes by thresholding on object and class confidence.
    
    Arguments:
    box_confidence -- tensor of shape (19, 19, 5, 1)
    boxes -- tensor of shape (19, 19, 5, 4)
    box_class_probs -- tensor of shape (19, 19, 5, 80)
    threshold -- real value, if [ highest class probability score < threshold], 
    then get rid of the corresponding box
    
    Returns:
    scores -- tensor of shape (None,), containing the class probability score for selected boxes
    boxes -- tensor of shape (None, 4), containing (b_x, b_y, b_h, b_w) coordinates of 
    selected boxes
    classes -- tensor of shape (None,), containing the index of the class detected by the 
    selected boxes
    
    Note: "None" is here because you don't know the exact number of selected boxes, 
    as it depends on the threshold. 
    For example, the actual output size of scores would be (10,) if there are 10 boxes.
    """
    
    # Step 1: Compute box scores by doing the element-wise product as shown in Fig. 4.
    box_scores = tf.multiply(box_confidence, box_class_probs)
    
    # Step 2:For each box, find the index of the class with the maximum box score
        # K.argmax gives back the index that has max value
        #axis=-1 means with respect to last dimension (the 85 )
    box_classes = K.argmax(box_scores, axis=-1) 
    
    # K.max gives back the max value with respect to the last dimension 
    box_class_scores = K.max(box_scores, axis=-1) 
    
    # Step 3: Create a filtering mask based on "box_class_scores" by using "threshold". 
    # As a reminder: ([0.9, 0.3, 0.4, 0.5, 0.1] < 0.4) returns: [False, True, False, False, True]. 
   # The mask should have the same dimension as box_class_scores,
    # and be True for the boxes you want to keep (with probability >= threshold)
    
    filtering_mask = K.greater_equal(box_class_scores, threshold)
    
    # Step 4: Use TensorFlow to apply the mask to box_class_scores, boxes and box_classes 
    #to filter out the boxes we don't want and leave only a subset of boxes we want to keep. 
    
    scores = tf.boolean_mask(box_class_scores, filtering_mask)
    boxes = tf.boolean_mask(boxes, filtering_mask)
    classes = tf.boolean_mask(box_classes, filtering_mask)
    
    return scores, boxes, classes

### 2.3  Filter 2 - Non-max suppression ###

Even after filtering by thresholding over the classes scores (Filter 1), there are still a lot of overlapping boxes. A second filter for selecting the right boxes is called non-maximum suppression (NMS). 
NMS selects only one box when several boxes overlap with each other and detect the same object.

<img src="nb_images/non-max-suppression.png" style="width:500px;height:400;">
<caption><center> <u> **Figure 7** </u>: In this example, the model has predicted 3 cars, but it's actually 3 predictions of the same car. Running non-max suppression (NMS) will select only the most accurate (highest probabiliy) one of the 3 boxes. <br> </center></caption>


Non-max suppression uses the function called **"Intersection over Union"**, or IoU.
<img src="nb_images/iou.png" style="width:500px;height:400;">
<caption><center> <u> **Figure 8** </u>: Definition of "Intersection over Union". <br> </center></caption>


The key steps of non-max suppression:
1. Select the box that has the highest score.
2. Compute its overlap with all other boxes, and remove boxes that overlap it more than `iou_threshold`.
3. Go back to step 1 and iterate until there's no more boxes with a lower score than the current selected box.

This will remove all boxes that have a large overlap with the selected boxes. Only the "best" boxes remain.

Two built-in functions of  TensorFlow are used to implement NMS in yolo_non_max_suppression()

- [tf.image.non_max_suppression()](https://www.tensorflow.org/api_docs/python/tf/image/non_max_suppression)
- [K.gather()](https://www.tensorflow.org/api_docs/python/tf/gather)

In [3]:
#FILTER 2

def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    """
    Applies Non-max suppression (NMS) to set of boxes
    
    Arguments:
    scores -- tensor of shape (None,), output of yolo_filter_boxes()
    boxes -- tensor of shape (None, 4), output of yolo_filter_boxes() that have been scaled to the image size (see later)
    classes -- tensor of shape (None,), output of yolo_filter_boxes()
    max_boxes -- integer, maximum number of predicted boxes you'd like
    iou_threshold -- real value, "intersection over union" threshold used for NMS filtering
    
    Returns:
    scores -- tensor of shape (, None), predicted score for each box
    boxes -- tensor of shape (4, None), predicted box coordinates
    classes -- tensor of shape (, None), predicted class for each box
    
    Note: The "None" dimension of the output tensors has to be less than max_boxes. 
    Note also that this function will transpose the shapes of scores, boxes, classes. 
    #This is made for convenience.
    """
    
    # tensor to be used below in tf.image.non_max_suppression()
    max_boxes_tensor = K.variable(max_boxes, dtype='int32')     
  
    # initialize variable max_boxes_tensor
    K.get_session().run(tf.variables_initializer([max_boxes_tensor])) 
    
    
    # Use tf.image.non_max_suppression() to get the list of indices corresponding to boxes to keep
    nms_indices = tf.image.non_max_suppression(boxes, scores, max_boxes, iou_threshold)

     
    # Use K.gather() to select only nms_indices from scores, boxes and classes
    scores = K.gather(scores,nms_indices)
    boxes = K.gather(boxes,nms_indices)
    classes = K.gather(classes,nms_indices)
   
    
    return scores, boxes, classes

### 2.4 Wrapping up the filtering

Function `yolo_eval()` takes the output of the YOLO deep CNN (the 19x19x5x85 dimensional encoding)  and filters the boxes using score threshold (`yolo_filter_boxes`) and NMS (`yolo_non_max_suppression`). 

There are a few ways of representing boxes, such as via their corners or via their midpoint and height/width. YOLO converts between a few such formats, using the following functions (here provided): 

```python
boxes = yolo_boxes_to_corners(box_xy, box_wh) 
```
which converts the yolo box coordinates (x,y,w,h) to box corners' coordinates (x1, y1, x2, y2) to fit the input of `yolo_filter_boxes`. 

YOLO's network was trained to run on 608x608 images. If you are testing this data on a different size image--for example, the car detection dataset had 720x1280 images--this code rescales the boxes so that they can be plotted on top of the original image_shape =720x1280.  

```python
boxes = scale_boxes(boxes, image_shape)

boxes consist of 4 numbers that represent the pixels in the points (0,0) and (1,1) of the detected box, but rescaled to fit the original image. 
```

In [4]:
def yolo_eval(yolo_outputs, image_shape = (720., 1280.), max_boxes=10, score_threshold=.6, iou_threshold=.5):
    """
    Converts the output of YOLO encoding (a lot of boxes) to your predicted boxes along with their scores, 
    box coordinates and classes.
    
    Arguments:
    yolo_outputs -- output of the encoding model (for image_shape of (608, 608, 3)), 
                    contains 4 tensors:
                    box_confidence: tensor of shape (None, 19, 19, 5, 1)
                    box_xy: tensor of shape (None, 19, 19, 5, 2)
                    box_wh: tensor of shape (None, 19, 19, 5, 2)
                    box_class_probs: tensor of shape (None, 19, 19, 5, 80)
                    
    image_shape -- tensor of shape (2,) containing the input shape, in this notebook we 
    use (608., 608.) (has to be float32 dtype)
    max_boxes -- integer, maximum number of predicted boxes you'd like
    score_threshold -- real value, if [ highest class probability score < threshold], 
    then get rid of the corresponding box
    iou_threshold -- real value, "intersection over union" threshold used for NMS filtering
    
    Returns:
    scores -- tensor of shape (None, ), predicted score for each box
    boxes -- tensor of shape (None, 4), predicted box coordinates
    classes -- tensor of shape (None,), predicted class for each box
    """
    
    # Retrieve outputs of the YOLO model
    box_confidence, box_xy, box_wh, box_class_probs = yolo_outputs

    # Convert boxes to be ready for filtering functions 
    boxes = yolo_boxes_to_corners(box_xy, box_wh)

    # Use Filter 1 to perform Score-filtering with a threshold of score_threshold
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold=score_threshold)
    
    # Scale boxes back to original image shape.

    boxes = scale_boxes(boxes, image_shape)

    # Use Filter 2 to perform Non-max suppression with a threshold of iou_threshold
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes=max_boxes, iou_threshold=iou_threshold)
    
    return scores, boxes, classes

**Summary for YOLO**:
- Input image (608, 608, 3)
- The input image goes through a CNN, resulting in a (19,19,5,85) dimensional output. 
- After flattening the last two dimensions, the output is a volume of shape (19, 19, 425):
    - Each cell in a 19x19 grid over the input image gives 425 numbers. 
    - 425 = 5 x 85 because each cell contains predictions for 5 boxes, corresponding to 5 anchor boxes. 
    - 85 = 5 + 80 where 5 is because $(p_c, b_x, b_y, b_h, b_w)$ has 5 numbers, and 80 is the number of classes to detect
- Select only few boxes based on:
    - Score-thresholding: throw away boxes that have detected a class with a score less than the threshold
    - Non-max suppression: Compute the Intersection over Union and avoid selecting overlapping boxes
- This gives YOLO's final output. 

## 3 - Test YOLO pretrained model on images

In this part, you are going to use a pretrained model and test it on the car detection dataset. 
Tensorflow requires first to create a session, as the cell below. 

In [5]:
sess = K.get_session()

### 3.1 - Define classes, anchors and image shape

Information about the 80 classes and the 5 boxes is gathered in the files "coco_classes.txt" and "yolo_anchors.txt". These quantities are loaded next.  

In [6]:
class_names = read_classes("model_data/coco_classes.txt")
anchors = read_anchors("model_data/yolo_anchors.txt")   

### 3.2 - Load YOLO pretrained model

Training a YOLO model takes a very long time and requires a fairly large dataset of labelled bounding boxes for a large range of target classes. In this lab you will load an existing pretrained Keras YOLO model stored in "yolo.h5". (These weights come from the official YOLO website, and were converted using a function written by Allan Zelener. References are given at the begining of this notebook. Technically, these are the parameters from the "YOLOv2" model, but we will more simply refer to it as "YOLO" in this notebook.) Run the cell below to load the model from this file.

**Note**: On some computers, you may see a warning message from Keras. Don't worry about it.

In [7]:
warnings.filterwarnings('ignore',category=DeprecationWarning)

yolo_model = load_model("model_data/yolo.h5")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.




/home/gabriel/.local/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


This loads the weights of a trained YOLO model. Here's a summary of the layers your model contains.

In [8]:
yolo_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 608, 608, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 608, 608, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 608, 608, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 608, 608, 32) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

### 3.3 - Convert output of the model to usable bounding box tensors

 `yolo_model` converts a preprocessed batch of input images (shape: (m, 608, 608, 3)) into an output tensor of shape (m, 19, 19, 5, 85). 

Function `yolo_head` gives back yolo_outputs to be used in `yolo_eval` function.

In [9]:
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))

### 3.4 - Filtering boxes

Run `yolo_eval` to perform filtering and select only the best boxes.

The car detection dataset has 720x1280 images, which has to be first pre-processed into 608x608 images.

In [10]:
image_shape = (720.,1280.)

scores, boxes, classes = yolo_eval(yolo_outputs, image_shape=image_shape)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### 3.5 - Run the graph on an image

The TensorFlow session graph (`sess`) can be summarized as follows:

1. <font color='purple'> yolo_model.input </font> is given to `yolo_model`. The model is used to compute the output <font color='purple'> yolo_model.output </font>
2. <font color='purple'> yolo_model.output </font> is processed by `yolo_head`. It gives you <font color='purple'> yolo_outputs </font>
3. <font color='purple'> yolo_outputs </font> goes through a filtering function, `yolo_eval`. It outputs your predictions: <font color='purple'> scores, boxes, classes </font>

Function ``predict()` below runs the graph to test YOLO on an image.
It runs inside a TensorFlow session, to compute `scores, boxes, classes`.

The code below also uses the following function:
```python
image, image_data = preprocess_image("images/" + image_file, model_image_size = (608, 608))
```
which outputs:
- image: a python (PIL) representation of your image used for drawing boxes. You won't need to use it.
- image_data: a numpy-array representing the image. This will be the input to the CNN.



In [11]:
def predict(sess, image_file):
    """
    Runs the graph stored in "sess" to predict boxes for "image_file". Prints and plots the preditions.
    
    Arguments:
    sess -- your tensorflow/Keras session containing the YOLO graph
    image_file -- name of an image stored in the "images" folder.
    
    Returns:
    out_scores -- tensor of shape (None, ), scores of the predicted boxes
    out_boxes -- tensor of shape (None, 4), coordinates of the predicted boxes
    out_classes -- tensor of shape (None, ), class index of the predicted boxes
    
    Note: "None" represents the number of predicted boxes, it varies between 0 and max_boxes. 
    """

    # Preprocess the input image
    #The car detection dataset has 720x1280 images, which has to be pre-processed into 608x608 images.
    
    image, image_data = preprocess_image("images/" + image_file, model_image_size = (608, 608))
    
    #When a model uses BatchNorm (as is the case in YOLO), you will need to pass an additional 
    #placeholder in the feed_dict {K.learning_phase(): 0}.
    
    out_scores, out_boxes, out_classes = sess.run([scores, boxes, classes],
                            feed_dict={yolo_model.input: image_data, K.learning_phase(): 0})

    # Print predictions info
    print('Found {} boxes for {}'.format(len(out_boxes), image_file))
    # Generate colors for drawing bounding boxes.
    colors = generate_colors(class_names)
    # Draw bounding boxes on the image file
    draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors)
    # Save the predicted bounding box on the image
    image.save(os.path.join("out", image_file), quality=90)
    # Display the results in the notebook
    output_image = scipy.misc.imread(os.path.join("out", image_file))
    imshow(output_image)
    
    return out_scores, out_boxes, out_classes

Run `predict` to observe object detection on "test.jpg" image.

In [12]:
out_scores, out_boxes, out_classes = predict(sess, "test.jpg")

UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv2d_1/convolution (defined at /home/gabriel/.local/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]
	 [[embedding_lookup_1/Identity/_713]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv2d_1/convolution (defined at /home/gabriel/.local/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'conv2d_1/convolution':
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ga

**Expected Output**:

<table>
    <tr>
        <td>
            **Found 7 boxes for test.jpg**
        </td>
    </tr>
    <tr>
        <td>
            **car**
        </td>
        <td>
           0.60 (925, 285) (1045, 374)
        </td>
    </tr>
    <tr>
        <td>
            **car**
        </td>
        <td>
           0.66 (706, 279) (786, 350)
        </td>
    </tr>
    <tr>
        <td>
            **bus**
        </td>
        <td>
           0.67 (5, 266) (220, 407)
        </td>
    </tr>
    <tr>
        <td>
            **car**
        </td>
        <td>
           0.70 (947, 324) (1280, 705)
        </td>
    </tr>
    <tr>
        <td>
            **car**
        </td>
        <td>
           0.74 (159, 303) (346, 440)
        </td>
    </tr>
    <tr>
        <td>
            **car**
        </td>
        <td>
           0.80 (761, 282) (942, 412)
        </td>
    </tr>
    <tr>
        <td>
            **car**
        </td>
        <td>
           0.89 (367, 300) (745, 648)
        </td>
    </tr>
</table>

The model you've run is able to detect 80 different classes listed in "coco_classes.txt". Test the model on your own images:

If you were to run your session in a for loop over all your images. Here's what you would get:

<center>
<video width="400" height="200" src="nb_images/pred_video_compressed2.mp4" type="video/mp4" controls>
</video>
</center>

<caption><center> Predictions of the YOLO model on pictures taken from a camera while driving around the Silicon Valley <br> (provided by [drive.ai](https://www.drive.ai/)) </center></caption>